In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from gensim.models import Word2Vec
import random
import pickle

E:\Philone\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#Extracting key-word of each article of X
def key_extract(X,Y,weights,spilt_factor=0.1,print_detail = 0):  
    ret_X = []                         #save the key-word of all article 
    for i in range(len(X)):
        words = X[i]                  #each word in article-i 
        temp_X = []                     #save the key-word repeated of an article 
        for word in words:
            temp = weights[int(word)][Y[i]]
            if temp > spilt_factor:
                temp_X.append(word)     #save the word
        if i<print_detail:
            print('Line'+str(i)+': total word: '+str(len(words))+' key word(repition): '+str(len(temp_X))+' percentage: '+str(len(temp_X)*1./len(words)))
            print(temp_X)
        ret_X.append(temp_X)             #save the article
    return ret_X

In [3]:
def word_count(X):
    words = set()
    for x in X:
        for word in x:
            words.add(word)
    return len(words),words

In [4]:
def article_count(X,key_word,print_detail = 0):
    X_key_word = []   # key-word repeated of all articles
    no_word_article = []   #sum of article without key-word 
    for i in range(len(X)):
        x_key_word = []    #key-word repeated of an article
        dif_word = set()
        for word in X[i]:
            if word in key_word:
                x_key_word.append(word)
                dif_word.add(word)
        if len(dif_word) == 0:
            no_word_article.append(i)
        X_key_word.append(x_key_word)
        if i<print_detail:
            print('line '+str(i)+' existing word: '+str(len(x_key_word))+' '+str(len(x_key_word)*1./len(X[i]))+' different word: '+str(len(dif_word))+' '+str(len(dif_word)*1./len(X[i])))
    return no_word_article,X_key_word

In [39]:
def article_extract_compliment(X,key_word):
    X_key_word = []   # key-word repeated of all articles
    for i in range(len(X)):
        x_key_word = []    #key-word repeated of an article
        no_word = True
        for word in X[i]:
            if word in key_word:
                x_key_word.append(word)
                no_word = False
        if no_word:
            for j in range(100):
                k = random.randint(0,len(X[i])-1)
                x_key_word.append(X[i][k])
        X_key_word.append(x_key_word)
    return X_key_word

In [6]:
def choose_weights(X,Y,weights1,weights1_split_factor,weights2,weights2_split_factor):
    log = {'num_of_key_word_after_weights1':0,'num_of_key_word_after_weights2':0,'no_word_article':0}
    
    weights_X = key_extract(X,Y,weights1,weights1_split_factor)    #using weights1 to get key-word
    log['num_of_key_word_after_weights1'],_ = word_count(weights_X)
    
    weights_X = key_extract(weights_X,Y,weights2,weights2_split_factor)  #using weights2 to get key-word
    log['num_of_key_word_after_weights2'],key_word_set = word_count(weights_X)
    
    log['no_word_article'],_ = article_count(X,key_word_set,print_detail = 0)  #counting the missing articles
    return log

In [7]:
#generator version:
def choose_weights_generator(X,Y,weights1,weights1_split_factor,weights2):
    log = {'num_of_key_word_after_weights1':0,'num_of_key_word_after_weights2':0,'no_word_article':0}
    
    weights1_X = key_extract(X,Y,weights1,weights1_split_factor)    #using weights1 to get key-word
    log['num_of_key_word_after_weights1'],_ = word_count(weights1_X)
    for j in range(11):
        log['num_of_key_word_after_weights2'] = 0
        log['no_word_article'] = 0
        weights2_X = key_extract(weights1_X,Y,weights2,j/10.)  #using weights2 to get key-word
        log['num_of_key_word_after_weights2'],key_word_set = word_count(weights2_X)
    
        log['no_word_article'],_ = article_count(X,key_word_set,print_detail = 0)  #counting the missing articles
        yield log

In [8]:
train_set = pd.read_csv('./../data/train_set.csv')
weights1 = np.load('./../data/word_weigt.npy')
weights2 = np.load('./../data/word_article.npy')

In [9]:
pre_X = train_set['word_seg'].values
Y = train_set['class'].values
X = []
for x in pre_X:
    X.append(x.split())

In [10]:
ret_orig_X = key_extract(X,Y,weights1,0.3)
print(len(ret_orig_X))
word_num,_ = word_count(ret_orig_X)
print(word_num)

102277
817383


In [11]:
ret_orig_X = key_extract(ret_orig_X,Y,weights2,0.1)
print(len(ret_orig_X))
word_num,key_word = word_count(ret_orig_X)
print(word_num)

102277
2742


In [40]:
train_X = article_extract_compliment(X,key_word)
f=open('./../data/train_X.pkl','wb')  
pickle.dump(train_X,f,0)  
f.close()

In [45]:
j=0
for i in train_X:
    if(len(i) == 100):
        j+=1
print(j)

1246


In [ ]:
print(len(no_word_article))

missing_Y = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for each in no_word_article:
    missing_Y[Y[each]] +=1
    
print(missing_Y)

In [ ]:
print(choose_weights(X,Y,weights1,0.4,weights2,0.1))

In [ ]:
for i in range(11):
     for j in range(11):
        log = choose_weights_generator(X,Y,weights1,i/10.,weights2,j/10.)
        print('('+str(i/10.)+','+str(j/10.)+'):',end = '')
        print(log)

In [ ]:
for i in range(11):
    j = 0
    for log in choose_weights_generator(X,Y,weights1,i/10.,weights2):
        print('('+str(i/10.)+','+str(j/10.)+'):',end = '')
        print(log)
        j+=1